# Section 1 : Basic Chat Interface

In [1]:
from dotenv import load_dotenv
import os
import requests
from bs4 import BeautifulSoup
import gradio as gr
from openai import OpenAI
import anthropic
import google.generativeai

In [2]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyCp


In [3]:
openai= OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()

In [11]:
system_prompt ='You are a helpful assistant'

In [17]:
def chat(message,history):
    messages = [{'role':'system','content':system_prompt}]
    for user_message,assistant_message in history:
        messages.append({'role':'user','content':user_message})
        messages.append({'role':'assistant','content':assistant_message})
    messages.append({'role':'user','content':message})

    stream = openai.chat.completions.create(
        messages= messages,
        model = 'gpt-4o-mini',
        stream=True
    )
    response =''
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace('```','').replace('markdown','')
        yield response

In [19]:
gr.ChatInterface(fn = chat,type = 'messages').launch(share=True)

* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://149e82b3a7652a15db.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Section 2 : Personalized Chat Interface

In [4]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get.\
Another example is , if the customer says 'I am looking to buy something for a gift', \
You can reply them something like, 'Excellent - we have variety of t-shirt and pant combinations that are available which are\
are also part of our sales event.' Encourage the customer to buy something from the store."

In [63]:
def chat(message,history):
    messages = []
    for user_message, assistant_message, *rest in history:
        messages.append({'role':'user','content':user_message})
        messages.append({'role':'assistant','content':assistant_message})
    messages.append({'role':'user','content':message})

    response = claude.messages.stream(
        max_tokens = 1000,
        model = 'claude-sonnet-4-20250514',
        messages = messages,
        system = system_message
    )
    result =''
    with response as stream:
        for text in stream.text_stream:
            result += text or ''
            yield result

In [64]:
gr.ChatInterface(fn = chat,type = 'messages').launch(share=True)

* Running on local URL:  http://127.0.0.1:7877
* Running on public URL: https://4ee9b4bf09cb7b79a7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [5]:
system_message +="\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [33]:
gr.ChatInterface(fn = chat, type = 'messages').launch(share=True)

* Running on local URL:  http://127.0.0.1:7868
* Running on public URL: https://899b4ece9e3ca1a715.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Personalized Chat Interface with Multiple models

In [49]:
def chat_gpt(message,history):
    messages = [{'role':'system','content':system_message}]
    for user_message, assistant_message, *rest in history:
        messages.append({'role':'user','content':user_message})
        messages.append({'role':'assistant','content':assistant_message})
    messages.append({'role':'user','content':message})

    stream = openai.chat.completions.create(
        messages = messages,
        model = 'gpt-4o-mini',
        stream = True
    )
    response = ''
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace('```','').replace('markdown','')
        yield response

In [53]:
def chat_claude(message,history):
    messages = []
    for user_message,assistant_message , *rest in history:
        messages.append({'role':'user','content':user_message})
        messages.append({'role':'assistant','content':assistant_message})
    messages.append({'role':'user','content':message})

    response = claude.messages.stream(
        max_tokens = 10000,
        system = system_message,
        model ='claude-sonnet-4-20250514',
        messages = messages
    )
    result =''
    with response as stream:
        for text in stream.text_stream:
            result += text or ''
            yield result

In [55]:
def chat_gemini(message,history):
    messages = []
    for user_message, assistant_message, *rest in history:
        messages.append({'role': 'user', 'parts': [{'text': user_message}]})
        messages.append({'role': 'model', 'parts': [{'text': assistant_message}]})
    messages.append({'role': 'user', 'parts': [{'text': message}]})

    gemini = google.generativeai.GenerativeModel(
        system_instruction = system_message,
        model_name = 'gemini-2.5-flash-lite'
    )
    result =''
    response = gemini.generate_content(messages, stream = True)
    for chunk in response:
        if chunk.text:
            result += chunk.text 
            yield result

In [56]:
def chat(message,history,model):
    if model == 'GPT':
        response = chat_gpt(message,history)
    elif model == 'Claude':
        response = chat_claude(message,history)
    elif model == 'Gemini':
        response = chat_gemini(message,history)
    else:
        raise ValueError('Model Not Found')

    yield from response

In [60]:
gr.ChatInterface(fn = chat, type = 'messages',additional_inputs=gr.Dropdown(["GPT", "Claude", "Gemini"],label ='Select Your Model')).launch(share=True)

* Running on local URL:  http://127.0.0.1:7875
* Running on public URL: https://c896db1cd4a007673b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Use of System Prompt again after using once

In [66]:
def chat(message,history):
    messages = [{'role':'system','content':system_message}]
    for user_message, assistant_message , *rest in history:
        messages.append({"role":'user','content':user_message})
        messages.append({'role':'assistant','content':assistant_message})

    if 'belt' in message:
        messages.append({'role':'system','content':'The store does not sell belts; if you are asked for belts, be sure to point out other items on sale.'})

    messages.append({'role':'user','content':message})

    stream = openai.chat.completions.create(
        messages = messages,
        model = 'gpt-4o',
        stream = True
    )
    response =''
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace('```','').replace('markdown','')
        yield response

In [67]:
gr.ChatInterface(fn = chat, type = 'messages').launch(share=True)

* Running on local URL:  http://127.0.0.1:7878
* Running on public URL: https://ead9a33dd3d538b97f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Well Informed Chat Interface : Prices Feed to the LLM

In [18]:
def chat(message,history):
    messages = [{'role':'system','content':system_message}]
    for user_message,assistant_message, * rest in history:
        messages.append({'role':'user','content':user_message})
        messages.append({'role':'assistant','content':assistant_message})
        
    price = {
        'small hat' : '20$',
        'medium hat' : '25$',
        'large hat' : '30$'
    }

    
    if isinstance(message, str) and message.lower() in ['price', 'cost', 'fee', 'retail', 'value', 'amount', 'listing', 'worth']:
        messages.append({'role':'system','content':f"Product prices are decided based on size. \
        Here are the before-sale prices: {price}. \
        Always provide both the before-sale price and \
        the after-sale price (apply any discount mentioned by the user). \
        If no discount is specified, show only the before-sale price."})
        
    messages.append({'role':'user','content':message})

    stream = openai.chat.completions.create(
        messages = messages,
        model = 'gpt-4o-mini',
        stream = True
    )

    response = ''
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace('```','').replace('markdown','')
        yield response

In [19]:
gr.ChatInterface(fn = chat,type='messages').launch(share=True)

* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://10c9bd3665de5fe889.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
